# HW 4 Programming

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
dim = 20
N_test = int(1e5)
Ns = [1000, 10000, 100000, 1000000]

Your task is to fill out the methods in the cell below.

Then, run the rest of the cells in this notebook and insert the plots into your homework.

In [ ]:
def t_learner(X, Y, T, X_test):
    # Train T-Learner based on training data X, Y, and T (see pseudocode)
    # Returns tau_hat, a vector of size X_test.shape[0]
    # with tau_hat[i] being the estimate of E[Y^1 - Y^0|X=X_test[i,:]]
    
    return tau_hat

def s_learner(X, Y, T, X_test):
    # Train S-Learner based on training data X, Y, and T (see pseudocode)
    # Returns tau_hat, a vector of size X_test.shape[0]
    # with tau_hat[i] being the estimate of E[Y^1 - Y^0|X=X_test[i,:]]
    
    return tau_hat

def x_learner(X, Y, T, g, X_test):
    # Train X-Learner based on training data X, Y, T, and g (see pseudocode)
    # Returns tau_hat, a vector of size X_test.shape[0]
    # with tau_hat[i] being the estimate of E[Y^1 - Y^0|X=X_test[i,:]]
    
    return tau_hat

In [ ]:
def run_experiment(N_train, e, get_mu):
    mu_0, mu_1 = get_mu()

    # Covariance matrix of data (random)
    r = np.random.random((dim,dim+1))*5
    Sigma = np.cov(r)

    X = np.random.multivariate_normal(np.zeros(dim), Sigma, size=N_train+N_test)
    W = np.random.random(N_train) < e(X[:N_train])

    Y = np.zeros(N_train)
    for i in range(N_train):
        Y[i] = (mu_0(X[i]) if W[i] == 0 else mu_1(X[i])) + np.random.normal()

    cate = np.zeros(N_test)
    for i in range(N_test):
        cate[i] = mu_1(X[N_train+i,:]) - mu_0(X[N_train+i,:])

    slearner_tau = s_learner(X[:N_train],Y,W,X[-N_test:])
    tlearner_tau = t_learner(X[:N_train],Y,W,X[-N_test:])
    xlearner_tau = x_learner(X[:N_train],Y,W,e,X[-N_test:])

    s_err = np.sqrt(np.mean(np.power(slearner_tau - cate,2)))
    t_err = np.sqrt(np.mean(np.power(tlearner_tau - cate,2)))
    x_err = np.sqrt(np.mean(np.power(xlearner_tau - cate,2)))

    return {
        's_err': s_err,
        't_err': t_err,
        'x_err': x_err,
    }

In [ ]:
def all_experiments(e, get_mu):
    results = {
      's_err': [],
      't_err': [],
      'x_err': [],
    }

    for N in Ns:
        s_errs = []
        t_errs = []
        x_errs = []
        for _ in range(30):
            result = run_experiment(N, e, get_mu)
            s_errs.append(result['s_err'])
            t_errs.append(result['t_err'])
            x_errs.append(result['x_err'])
        results['s_err'].append(np.mean(s_errs))
        results['t_err'].append(np.mean(t_errs))
        results['x_err'].append(np.mean(x_errs))
        print(f"Finish running with {N} training samples.")

    plt.figure()
    plt.plot(np.log10(Ns), np.log10(results['s_err']), 'go-', label='S−learner')
    plt.plot(np.log10(Ns), np.log10(results['t_err']), 'k^-', label='T−learner')
    plt.plot(np.log10(Ns), np.log10(results['x_err']), 'b+-', label='X−learner')
    plt.xlabel('Log10(size of training data)')
    plt.ylabel('Log10(MSE)')
    plt.title('CATE loss')
    plt.legend()
    plt.grid()
    plt.show()

## Part (a)

In [ ]:
def e(x):
    return 0.01

def get_mu():
    beta = np.random.uniform(low=-5, high=5, size=dim)
    
    def mu_0(x):
        return np.dot(x, beta)

    def mu_1(x):
        return mu_0(x) + 8 * x[0]
    
    return mu_0, mu_1

all_experiments(e, get_mu)

## Part (b)

In [ ]:
def e(x):
    return 0.5

def get_mu():
    beta0 = np.random.uniform(low=-5, high=5, size=dim)
    beta1 = np.random.uniform(low=-5, high=5, size=dim)
    
    def mu_0(x):
        return np.dot(x, beta0)

    def mu_1(x):
        return np.dot(x, beta1)
    
    return mu_0, mu_1

all_experiments(e, get_mu)

## Part (c)

In [ ]:
def e(x):
    return 0.5

def get_mu():
    beta0 = np.random.uniform(low=-5, high=5, size=dim)
    
    def mu_0(x):
        return np.dot(x, beta0)

    def mu_1(x):
        return mu_0(x)
    
    return mu_0, mu_1

all_experiments(e, get_mu)

## Part (d)

In [ ]:
def e(X):
    return 0.5 + 0.25 * np.sign(X[:,0])

def get_mu():
    beta0 = np.random.uniform(low=-5, high=5, size=dim)
    
    def mu_0(x):
        return 2 * x[0] + 5

    def mu_1(x):
        return mu_0(x)
    
    return mu_0, mu_1

all_experiments(e, get_mu)